In [3]:
import pandas as pd

State_of_data_BR_2023 = pd.read_csv("../../data/processed/stateOfDataBR2023_tratado.csv", low_memory=False)

df = pd.DataFrame(State_of_data_BR_2023)

def calcular_valor_coluna(merged):
  merged['total_percentual'] = (merged['total'] / merged['total_nivel']) * 100
  return merged

#   O código cria um novo DataFrame chamado 'consulta' que contém apenas as linhas que atendem aos critérios
#  de filtragem especificados, mantendo apenas as colunas de interesse.
consulta = df[
       (df['(P2_g_-_Nivel)'].isin(['Junior', 'Pleno', 'Senior'])) &
    (df['(P2_t_-_Caso_sua_empresa_decida_pelo_modelo_100%_presencial_qual_sera_sua_atitude?)'].isin([
'Vou aceitar e retornar ao modelo 100% presencial','Vou procurar outra oportunidade no modelo 100% remoto','Vou procurar outra oportunidade no modelo hibrido ou remoto'])) &

    # Código padrão abaixo.
    (df['(P2_a_-_Qual_sua_situacao_atual_de_trabalho?)'].ne('Desempregado, buscando recolocacao')) &
    (df['(P2_a_-_Qual_sua_situacao_atual_de_trabalho?)'].ne('Desempregado e nao estou buscando recolocacao')) &
    (df['(P2_a_-_Qual_sua_situacao_atual_de_trabalho?)'].ne('Somente Estudante (graduacao)')) &
    (df['(P2_a_-_Qual_sua_situacao_atual_de_trabalho?)'].ne('Prefiro nao informar'))
    # Código padrão acaba aqui.
    ][
        [
         '(P2_g_-_Nivel)',
         '(P2_t_-_Caso_sua_empresa_decida_pelo_modelo_100%_presencial_qual_sera_sua_atitude?)',
         '(P2_a_-_Qual_sua_situacao_atual_de_trabalho?)' ]]

# # Agrupando por nível e gênero e somando os totais
decisao_por_cargo = consulta.groupby('(P2_g_-_Nivel)')['(P2_t_-_Caso_sua_empresa_decida_pelo_modelo_100%_presencial_qual_sera_sua_atitude?)'].value_counts().reset_index(name='total')

# Calculando o total por nível
totais_por_nivel = decisao_por_cargo.groupby(['(P2_g_-_Nivel)','(P2_t_-_Caso_sua_empresa_decida_pelo_modelo_100%_presencial_qual_sera_sua_atitude?)'])['total'].sum().reset_index()

# Renomeando a coluna 'total' para 'total_nivel'
totais_por_nivel = decisao_por_cargo.groupby('(P2_g_-_Nivel)')['total'].sum().reset_index()
totais_por_nivel.rename(columns={'total': 'total_nivel'}, inplace=True)

# # Juntando os totais por nível com o DataFrame agrupado
merged = pd.merge(decisao_por_cargo, totais_por_nivel, on='(P2_g_-_Nivel)')

# # Calculando a porcentagem
merged['percentual'] = (merged['total'] / merged['total_nivel']) * 100
calcular_valor_coluna(merged)



# # Exibindo o resultado final
print(merged[['(P2_g_-_Nivel)','(P2_t_-_Caso_sua_empresa_decida_pelo_modelo_100%_presencial_qual_sera_sua_atitude?)','total', 'total_percentual']])


  (P2_g_-_Nivel)  \
0         Junior   
1         Junior   
2         Junior   
3          Pleno   
4          Pleno   
5          Pleno   
6         Senior   
7         Senior   
8         Senior   

  (P2_t_-_Caso_sua_empresa_decida_pelo_modelo_100%_presencial_qual_sera_sua_atitude?)  \
0  Vou procurar outra oportunidade no modelo hibr...                                    
1   Vou aceitar e retornar ao modelo 100% presencial                                    
2  Vou procurar outra oportunidade no modelo 100%...                                    
3  Vou procurar outra oportunidade no modelo hibr...                                    
4  Vou procurar outra oportunidade no modelo 100%...                                    
5   Vou aceitar e retornar ao modelo 100% presencial                                    
6  Vou procurar outra oportunidade no modelo hibr...                                    
7  Vou procurar outra oportunidade no modelo 100%...                                   

In [ ]:
import plotly.express as px
grafico = px.bar(merged, x='(P2_g_-_Nivel)', y='total_percentual', color='(P2_t_-_Caso_sua_empresa_decida_pelo_modelo_100%_presencial_qual_sera_sua_atitude?)',  title='Modalidade de Trabalho Por Cargo', barmode='group')
grafico.show()